In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
import numpy as np
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, LongformerForSequenceClassification, LongformerTokenizer, DebertaV2ForSequenceClassification
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments, PreTrainedTokenizer
from sklearn.model_selection import train_test_split
from collections import Counter
from transformers import BertModel
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm import tqdm
import torch.nn.functional as F
from types import SimpleNamespace
import re
from torch.optim import AdamW
from sklearn.model_selection import KFold
import os
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
import transformers
import gc

In [15]:
CONFIG = {
    "data_base": "../data"
}

In [16]:
train_csv = pd.read_csv(f"{CONFIG['data_base']}/pseudo_labeling.csv")
test_csv = pd.read_csv(f"{CONFIG['data_base']}/test.csv")

In [ ]:
train_csv = train_csv.rename(columns={
    'paragraph': 'full_text',
    'paragraph_label': 'generated'
    })

train_csv.head()

,Unnamed: 0.1,Unnamed: 0,title,paragraph_idx,full_text,generated,document_label
0,0,0,카호올라웨섬,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,0
1,1,1,카호올라웨섬,1,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...,0,0
2,2,2,카호올라웨섬,2,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...,0,0
3,3,3,카호올라웨섬,3,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...,0,0
4,4,4,카호올라웨섬,4,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...,0,0


In [18]:
label_0 = train_csv[train_csv['generated'] == 0] # 0 or 1
label_0 = label_0.sample(frac=1, random_state=42).reset_index(drop=True)
label_0

,Unnamed: 0.1,Unnamed: 0,title,paragraph_idx,full_text,generated,document_label
0,957376,958424,서울 북한산 구기동 마애여래좌상,0,서울 북한산 구기동 마애여래좌상(서울 北漢山 舊基洞 磨崖如來坐像)은 북한산 중턱에 ...,0,0
1,780693,781574,나루히토,0,어릴 적 칭호는 히로노미야. 그의 도장은 가래나무이다. 현재 재위하고 있음에 따라 ...,0,0
2,794964,795852,밀도범함수 이론,6,밀도범함수의 출발은 원자핵+전자 시스템을 풀기위해 시작됐지만 이론상 많은 동일한 형...,0,0
3,563426,564097,미국의 역사 (1789-1849),50,1833년 9월 잭슨은 정부의 돈을 향후 은행에 예탁하지 않고 이미 예탁된 것은 정...,0,0
4,823572,824495,이소 (굴원),0,《이소》(離騷)는 중국 전국 시대 초나라의 시인인 굴원이 지은 장편 서정시이다. 수...,0,0
...,...,...,...,...,...,...,...
1181813,114008,114091,형식,4,"형식이라는 뜻은 겉으로만 드러나는 것을 우선으로 하는 걸 의미하기도 한다. 예시로,...",0,0
1181814,268524,268779,일본의 법무대신,1,"일본의 검찰관은 각각이 검찰권을 행사하는 독임 관청이지만, 검찰권은 통일되어 행정권...",0,0
1181815,136676,136784,소울칼리버의 등장인물 목록,6,라파엘 / 타림 / 카산드라 / 홍윤성 / 샤레드 / 네크리드 / 어쌔신 / 버서커,0,0
1181816,695996,696836,비유클리드 기하학,5,유클리드의 다섯번째 공준인 평행선 공준은 다른 네 공준이 자와 컴퍼스를 이용해 경험...,0,0


In [19]:
model_id = "rtzr/ko-gemma-2-9b-it"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 3584, padding_idx=0)
    (layers): ModuleList(
      (0-41): 42 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
          (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
          (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((3584,), eps=1e-06)
        (pre_feedforward_layernorm): Gemma2RMSNorm((3584,), 

In [20]:
def prompt_gen(text):
    return (
        f"이 문장을 비슷한 문장으로 다시 작성해줘.\n"
        + f"다른 부수적인 말하지 말고 오직 비슷한 문장 1개만 말해주면 돼.\n"
        + f"문장: {text}"
    )

In [21]:
def generate(text, pipeline):
    messages = [
        {"role": "user", "content": prompt_gen(text)}
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<end_of_turn>")
    ]

    with torch.no_grad():  # memory leak 방지
        outputs = pipeline(
            prompt,
            max_new_tokens=2048,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )

    torch.cuda.empty_cache()  # GPU 캐시 비움
    return outputs[0]["generated_text"][len(prompt):]


In [23]:
augmentation = {
    "title" : [],
    "full_text" : [],
    "generated" : [],
}

for _, i in tqdm(label_0.iterrows(), total=len(label_0)):
    title = i['title']
    text = i['full_text']
    label = i['generated']

    aug_text = generate(text, pipeline)

    augmentation['title'].append(title)
    augmentation['full_text'].append(aug_text.strip().replace("\n", ""))
    augmentation['generated'].append(1)

    pd_aug = pd.DataFrame(augmentation)
    pd_aug.to_csv("./augmentation_data/aug_0.csv", index=False)

    gc.collect()
    torch.cuda.empty_cache()


  0%|          | 13/1181818 [00:36<914:53:57,  2.79s/it] 


KeyboardInterrupt: 